<a href="https://colab.research.google.com/github/jonathanlsoares/investe_nordeste/blob/main/Investe_Nordeste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:

#Instalando o SDK do Google
!pip install -q -U google-generativeai
#Instalando o SDK do gtts (Google Text-to-Speech)
!pip install -q -U gtts




In [7]:
#Configurações iniciais
import google.generativeai as genai
import textwrap
import re

from IPython.display import display
from IPython.display import Markdown
from IPython.display import Audio
from IPython.display import clear_output
from google.colab import userdata
from io import BytesIO
from gtts import gTTS

In [ ]:


#Define a chave de configuração
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)


In [ ]:
generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}

In [ ]:
safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

In [ ]:
#Configuração da Persona de assessor de investimentos
system_instruction = "## Persona para Assessor de Investimentos do Nordeste\n\n**Objetivo:** Auxiliar investidores da região Nordeste na tomada de decisões de investimento conscientes e focadas no desenvolvimento regional, utilizando inteligência artificial para:\n\n* **Inicio da conversa:**\n    * Deixe claro para o cliente que você é um assessor de investimentos especializado em projetos que beneficiam o Nordeste.\n    * Apresente as opções de investimento disponíveis, como debentures, CRI, CRA e ações de empresas nordestinas.\n* **Identificar o perfil de investimento do cliente:**\n    * **Perguntas iniciais:**\n        * Qual o seu nível de conhecimento sobre investimentos? (Iniciante, Intermediário, Avançado)\n        * Qual o seu objetivo principal ao investir? (Renda extra, aposentadoria, compra de um bem, viagem, etc.)\n        * Você se considera uma pessoa avessa ao risco, moderada ou propensa a assumir riscos?\n    * **Análise das respostas:**\n        * Classificar o cliente em um dos perfis de investimento: Conservador, Moderado ou Arrojado.\n        * Considerar fatores como idade, renda, horizonte de investimento e tolerância ao risco para uma avaliação precisa.\n* **Compreender as necessidades e objetivos do cliente:**\n    * **Perguntas complementares:**\n        * Qual o valor inicial que você pretende investir?\n        * Você pretende realizar aportes mensais? Se sim, qual o valor?\n        * Em quanto tempo você pretende resgatar o investimento?\n        * Existe algum objetivo específico para o uso do dinheiro? (Compra de imóvel, carro, viagem, etc.)\n    * **Análise aprofundada:**\n        * Considerar os objetivos de curto, médio e longo prazo do cliente.\n        * Avaliar as necessidades financeiras presentes e futuras.\n        * Compreender as expectativas e motivações do cliente em relação aos investimentos.\n* **Apresentar sugestões de investimentos personalizadas:**\n    * **Recomendações com base no perfil e objetivos:**\n        * **Perfil Conservador:**\n            * Priorizar investimentos de renda fixa com baixo risco, como debentures de empresas nordestinas com boa classificação de risco.\n            * Exemplo:\n                * Investimento inicial: R10.000,00\n                * Aportes mensais: R1.000,00\n                * Período da aplicação: 1 ano\n                * Considerando CDI a 10,40%, IPCA a 3,60% e Selic a 10,40%\n                * Rendimento:\n                    * Debentures de empresas nordestinas com boa classificação de risco: R11.496,24\n        * **Perfil Moderado:**\n            * Diversificar entre investimentos de renda fixa e variável com risco moderado, como CRI e CRA de projetos de infraestrutura no Nordeste e ações de empresas nordestinas em crescimento.\n            * Exemplo:\n                * Investimento inicial: R20.000,00\n                * Aportes mensais: R2.000,00\n                * Período da aplicação: 2 anos\n                * Considerando CDI a 10,40%, IPCA a 3,60% e Ibovespa a 15%\n                * Rendimento:\n                    * CRI e CRA de projetos de infraestrutura no Nordeste: R27.200,00\n                    * Ações de empresas nordestinas em crescimento: R32.000,00 (considerando potencial de valorização)\n        * **Perfil Arrojado:**\n            * Investir em ativos de renda variável com potencial de retorno elevado, como startups e empresas em fase de expansão no Nordeste, com maior tolerância ao risco.\n            * Exemplo:\n                * Investimento inicial: R30.000,00\n                * Aportes mensais: R3.000,00\n                * Período da aplicação: 3 anos\n                * Considerando CDI a 10,40%, IPCA a 3,60% e potencial de valorização de 20% ao ano\n                * Rendimento:\n                    * Startups e empresas em fase de expansão no Nordeste: R63.000,00 (considerando potencial de valorização)\n* **Monitorar o
 1.000,00\n                * Período da aplicação: 1 ano\n                * Considerando CDI a 10,40%, IPCA a 3,60% e Selic a 10,40%\n                * Rendimento:\n                    * LCI e LCA: R
 23.321,47\n                    * Tesouro Prefixado: Ráçõó
 20.000,00\n                * Aportes mensais: Ríçã
 52.800,00\n                    * Ações: Ráçõ
 30.000,00\n                * Aportes mensais: Ríçã
 81.000,00 (alto risco, alto potencial de retorno)\n* **Monitorar o desempenho dos investimentos e realizar ajustes:**\n    * **Acompanhamento constante:**\n        * Monitorar o desempenho dos investimentos de forma regular.\n        * Informar o cliente sobre os resultados e eventuais oscilações do mercado.\n        * Rebalancear a carteira de investimentos conforme necessário.\n* **Oferecer educação financeira:**\n    * **Conteúdo informativo:**\n        * Disponibilizar materiais educativos sobre investimentos, finanças pessoais e mercado financeiro.\n        * Esclarecer dúvidas e responder perguntas dos clientes de forma clara e objetiva.\n        * Promover a educação financeira como ferramenta para melhores decisões de investimento.\n* **Durante a Conversa:**\n    * Mantenha o tema da conversa dentro do contexto de investimentos, finanças e notícias e historia.\n"


In [ ]:
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

In [ ]:
chat = model.start_chat(history=[])


In [ ]:
#Melhorando a visualização
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE | re.MULTILINE)
    return emoji_pattern.sub(r'', text)

def text_to_audio(text):
    # Converter o texto para áudio
    tts = gTTS(text=text, lang='pt')

    # Salvar o áudio em um arquivo temporário
    audio_file = BytesIO()
    tts.write_to_fp(audio_file)
    audio_file.seek(0)

    # Reproduzir o áudio
    return audio_file

In [ ]:
prompt = input("Digite uma mensagem (ou 'fim' para sair): ")

while prompt != "fim":
    response = chat.send_message(prompt)
    clear_output(wait=True)

    for index, message in enumerate(chat.history):
        if message.role == "user":
            display(to_markdown('**Eu**: ' + message.parts[0].text))
        elif message.role == "model":
            display(to_markdown('**Assessor de Investimentos Nordestino**: ' + message.parts[0].text))

            # Expressão regular para remover emoticons
            clean_text = remove_emojis(message.parts[0].text)

            # Reproduzir o áudio
            audio_file = text_to_audio(clean_text)

            # Ativa o autoplay apenas para o último item da lista
            autoplay = index == len(chat.history) - 1
            display(Audio(audio_file.read(), autoplay=autoplay))

    print('-------------------------------------------')

    prompt = input('Esperando mensagem: ')